In [9]:
!pip install transformers==2.10.0

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import torch
torch.manual_seed(22)
from transformers import RobertaTokenizer
import numpy as np


In [12]:
df = pd.read_csv("/content/drive/My Drive/Reviews.csv")
MODEL_PATH = "/content/drive/My Drive/results/model_saved_1_14000.pt"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sentences = df.Text.values
labels = df.Score.values - 1

idx = np.where(labels==2)
#labels = np.delete(labels, idx)
#sentences = np.delete(sentences, idx)
labels = (labels>2).astype(int)
labels[idx] = 2
batch_size = 32
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)


In [13]:
input_ids = []
attention_masks = []
logits_list = []
# For every sentence...
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                 # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks. 
                        return_tensors = 'pt'   # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

In [14]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
print('phase score:', labels[2])
labels = torch.tensor(labels)

phase score: 1


In [15]:
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = torch.utils.data.random_split(dataset,[train_size, val_size])


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
test_dataloader = DataLoader(
            dataset, 
            sampler = SequentialSampler(dataset),
            batch_size = batch_size)

In [17]:
model = torch.load(MODEL_PATH).to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [18]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.where(pred_flat != labels_flat), np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
miss_classify_idx = []
total_eval_loss = 0
total_eval_accuracy = 0
logits_list = []
eval_data = test_dataloader
    # Evaluate data for one epoch
for step, batch in enumerate(eval_data):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
        
    with torch.no_grad():        
        (loss, logits) = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                               labels=b_labels)
            
        # Accumulate the validation loss.
    total_eval_loss += loss.item()

        # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
    idx, acc = flat_accuracy(logits, label_ids)
    logits_list.append(logits)
    if idx[0] != []:
      #print(idx[0])
      miss_idx = idx[0] +  int(step* batch_size)
      print(miss_idx)
      miss_classify_idx.append(miss_idx)
    
    total_eval_accuracy = total_eval_accuracy + acc

avg_acc = total_eval_accuracy / len(eval_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Streaming output truncated to the last 5000 lines.
[202795 202797 202802 202808 202814]
[202819 202831]
[202849 202856 202865 202875 202877]
[202945 202946 202954]
[203031 203034]
[203053 203055]
[203232 203257 203263]
[203274 203280 203289 203290]
[203380 203382]
[203570 203575 203576]
[203584 203588 203604]
[203622 203629 203635 203638]
[203656 203657 203665 203679]
[203690 203691 203709]
[203721 203724 203727]
[203744 203772]
[203857 203860 203863]
[203876 203894 203898]
[203904 203927 203934]
[203936 203937 203964]
[204085 204093]
[204104 204106 204114]
[204136 204139 204147 204153]
[204169 204183 204190]
[204320 204322 204327 204340]
[204402 204407]
[204429 204434]
[204515 204529 204541 204542]
[204611 204616 204625]
[204775 204776 204779 204790]
[204811 204812]
[204841 204862]
[205050 205052 205053 205054]
[205154 205172]
[205379 205381 205382 205386 205406]
[205475 205498]
[205510 205512 205513]
[205575 205578 205593]
[205605 205623]
[205644 205658 205662]
[205680 205695]
[20573

## Backup Functions

In [20]:
pred_labels = np.argmax(np.concatenate(logits_list),axis= 1).flatten()

In [21]:
idx_array = np.array(np.concatenate(miss_classify_idx))

In [22]:
#for e in idx_array:
 # print(e)
print(len(idx_array))

24167


In [23]:
import pickle 
with open("3classes.pkl" , "wb") as f:
  pickle.dump(idx_array, f)
  f.close()
with open("pred_3classes.pkl", "wb") as pred_f:
  pickle.dump(pred_labels, pred_f)
  pred_f.close()

In [24]:
total_eval_accuracy/len(test_dataloader)

0.9493509006473403